In [55]:
import requests
import json
import pandas as pd
import re

In [56]:
hdb_resale_data = pd.read_csv("data/processed/hdb_resale_data.csv")

In [57]:
def expand_abbreviation(street):
    street = " " + street + " "  # add padding to catch start/end abbreviations safely
    street = re.sub(r"\sAVE\s", " AVENUE ", street)
    street = re.sub(r"\sST\s", " STREET ", street)
    street = re.sub(r"\sRD\s", " ROAD ", street)
    return street.strip()

hdb_resale_data['street_name'] = hdb_resale_data['street_name'].apply(expand_abbreviation)

In [62]:
hdb_resale_data.head()

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,1,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVENUE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVENUE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,3,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVENUE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,4,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVENUE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,5,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVENUE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


In [70]:
for i in range(0, len(hdb_resale_data)):
    search_val = hdb_resale_data.iloc[i, 4] + " " + hdb_resale_data.iloc[i, 2]
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response = requests.get(url)

    for result in response.json()['results']:
        if result['ROAD_NAME'] == hdb_resale_data.iloc[i, 5]:
            hdb_resale_data.loc[i, 'latitude'] = result['LATITUDE']
            hdb_resale_data.loc[i, 'longitude'] = result['LONGITUDE']

KeyboardInterrupt: 

In [68]:
hdb_resale_data.head()

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude
0,1,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVENUE 4,07 TO 09,60.0,Improved,1986,70,255000.0,1.37509746867904,103.83761896123
1,2,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVENUE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,NaN,NaN
2,3,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVENUE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,NaN,NaN
3,4,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVENUE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,NaN,NaN
4,5,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVENUE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,NaN,NaN


In [39]:

# Address to search
search_val = "38 BEDOK STH ROAD"

# Format the search URL
url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"

# Send the request
response = requests.get(url)

# Print the result
print(json.dumps(response.json(), indent=2))

{
  "found": 1,
  "totalNumPages": 1,
  "pageNum": 1,
  "results": [
    {
      "SEARCHVAL": "BEDOK SOUTH PARKVIEW",
      "BLK_NO": "38",
      "ROAD_NAME": "BEDOK SOUTH ROAD",
      "BUILDING": "BEDOK SOUTH PARKVIEW",
      "ADDRESS": "38 BEDOK SOUTH ROAD BEDOK SOUTH PARKVIEW SINGAPORE 460038",
      "POSTAL": "460038",
      "X": "39760.7011717223",
      "Y": "33675.9521351345",
      "LATITUDE": "1.32082560856834",
      "LONGITUDE": "103.938994585574"
    }
  ]
}
